In [1]:
import numpy
import numpy as np
import pandas
import pandas as pd
from numba import njit
import sys,glob
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
sys.path.append(os.getcwd()+'/../cds-backend/code/')
sys.path.append(os.getcwd()+'/../harvest/code/')
from harvest_convert_to_netCDF_newfixes import write_dict_h5
import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time
#import schedule
import copy
from shutil import copyfile
import multiprocessing
sys.path.append(os.getcwd()+'/../resort/rasotools-master/')
import rasotools
np.seterr(all='warn')
np.errstate(divide='raise')

## Calculating the estimate time for calculation:

In [2]:
files = glob.glob('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/*.nc')
files[6020]

'/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-54102_CEUAS_merged_v0.nc'

In [3]:
b = os.path.getsize(files[6020])
b

122974841

In [4]:
t = 2633.3687977790833/60
t

43.88947996298472

v2:

In [5]:
tnew = (1306.7448754310608 + 185.2546329498291) / 60
tnew

24.86665847301483

In [6]:
b/tnew

4945370.570535308

Speed: ~4.9 MB/(min thread)

In [7]:
dirsize = 0 
for i in files:
    dirsize += os.path.getsize(i)
dirsize

618842993516

In [8]:
mintime = (dirsize)/(b/tnew)

In [9]:
mintime/60/24 , 'days'

(86.89987387214894, 'days')

~ 4 days on 20 cores -> more cores available on the jet?

---
---

## v1:

In [10]:
filesout = glob.glob('/raid60/scratch/uli/converted/*')
filesout

['/raid60/scratch/uli/converted/0-20000-0-22152_CEUAS_merged_v0.nc',
 '/raid60/scratch/uli/converted/0-20000-0-11019_CEUAS_merged_v0.nc',
 '/raid60/scratch/uli/converted/0-20000-0-54102_CEUAS_merged_v0.nc',
 '/raid60/scratch/uli/converted/0-20000-0-11035_CEUAS_merged_v0.nc']

In [11]:
# with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc') as test:
with eua.CDMDataset(filesout[2]) as test:

    print(test)
    print(test.recordindices)
    print(test.observations_table)
    print(test.era5fb)
    print(test.era5fb['an_depar@body'][:])
    print(test.era5fb['fg_depar@body'][:])
    aa = test.to_dataframe(groups=['observations_table', 'era5fb'], variables=['z_coordinate', 'date_time','observed_variable', 'observation_value', 'conversion_flag', 'conversion_method', 'fg_depar@body', 'an_depar@body','biascorr@body', 'biascorr_fg@body'])
aa

File: <HDF5 file "0-20000-0-54102_CEUAS_merged_v0.nc" (mode r)>
Filename: /raid60/scratch/uli/converted/0-20000-0-54102_CEUAS_merged_v0.nc
(G)roups/(V)ariables: 

 - G | crs__________________________________________ : : 4
 - V | dateindex____________________________________ : : (22851,)
 - G | era5fb_______________________________________ : : 71
 - G | header_table_________________________________ : : 56
 - G | observations_table___________________________ : : 49
 - G | observed_variable____________________________ : : 9
 - G | recordindices________________________________ : : 18
 - G | sensor_configuration_________________________ : : 12
 - G | source_configuration_________________________ : : 2
 - G | station_configuration________________________ : : 48
 - G | station_configuration_codes__________________ : : 7
 - G | station_type_________________________________ : : 4
 - G | units________________________________________ : : 6
 - G | z_coordinate_type____________________________ : : 

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
0,9473.223633,1964-06-01 11:00:00,0,90000.000000,-2147483648,-2147483648,-131.210556,-170.020813,0.0,0.0
1,9473.223633,1964-07-15 11:00:00,0,88500.000000,-2147483648,-2147483648,-271.075256,-353.135101,0.0,0.0
2,9473.223633,1964-07-30 11:00:00,0,98500.000000,-2147483648,-2147483648,8763.082031,8730.548828,0.0,0.0
3,9473.223633,1964-08-04 23:00:00,0,89300.000000,-2147483648,-2147483648,-244.882065,-286.890900,0.0,0.0
4,9473.223633,1964-08-15 23:00:00,0,89900.000000,-2147483648,-2147483648,-172.731949,-172.108704,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4954544,89810.000000,2019-12-28 23:15:24,140,1.169487,-2147483648,-2147483648,0.506251,0.767653,NaN,NaN
4954545,90780.000000,2019-12-29 11:16:12,140,2.630227,-2147483648,-2147483648,3.134859,2.860141,NaN,NaN
4954546,91280.000000,2019-12-29 23:15:47,140,0.615662,-2147483648,-2147483648,1.913419,2.676003,NaN,NaN
4954547,91770.000000,2019-12-30 11:15:22,140,4.405243,-2147483648,-2147483648,3.490680,3.252057,NaN,NaN


In [12]:
b_aa = aa[aa['conversion_flag'] == 0]
display(b_aa)
b_aa[np.isnan(b_aa['fg_depar@body'])]

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
41,30000.0,1956-08-20 15:00:00,34,9.89,0,3,5.820000,3.810000,NaN,NaN
42,40000.0,1956-08-20 15:00:00,34,9.69,0,3,1.500000,-2.720000,NaN,NaN
43,50000.0,1956-08-20 15:00:00,34,6.95,0,3,-18.830002,-21.860001,NaN,NaN
44,70000.0,1956-08-20 15:00:00,34,1.00,0,3,-5.600000,-8.800000,NaN,NaN
45,85000.0,1956-08-20 15:00:00,34,3.19,0,3,-3.310000,-4.140000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4251853,89810.0,2019-12-28 23:15:24,107,4.00,0,-2147483648,-0.767167,-0.605523,NaN,NaN
4251886,90780.0,2019-12-29 11:16:12,107,6.00,0,-2147483648,1.277103,1.391125,NaN,NaN
4251914,91280.0,2019-12-29 23:15:47,107,1.00,0,-2147483648,-3.204493,-3.804898,NaN,NaN
4251946,91770.0,2019-12-30 11:15:22,107,7.00,0,-2147483648,1.830617,1.644246,NaN,NaN


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
1371,81700.0,1957-06-22 00:00:00,34,5.25,0,3,NaN,4.47,NaN,NaN
1871,49900.0,1957-07-23 00:00:00,34,0.00,0,3,NaN,-0.19,NaN,NaN
1872,50000.0,1957-07-23 00:00:00,34,0.00,0,3,NaN,-0.17,NaN,NaN
1919,89200.0,1957-07-25 00:00:00,34,1.24,0,3,NaN,1.02,NaN,NaN
1951,82700.0,1957-07-27 00:00:00,34,0.00,0,3,NaN,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3700834,89810.0,2019-12-28 23:15:24,106,253.00,0,-2147483648,NaN,NaN,NaN,NaN
3700867,90780.0,2019-12-29 11:16:12,106,244.00,0,-2147483648,NaN,NaN,NaN,NaN
3700895,91280.0,2019-12-29 23:15:47,106,232.00,0,-2147483648,NaN,NaN,NaN,NaN
3700927,91770.0,2019-12-30 11:15:22,106,231.00,0,-2147483648,NaN,NaN,NaN,NaN


In [13]:
aa[np.isnan(aa['observation_value'])]

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
5419,36100.0,1958-04-27 00:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
32485,10000.0,1967-10-10 00:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
32486,20000.0,1967-10-10 00:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
32487,30000.0,1967-10-10 00:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
32493,10000.0,1967-10-11 00:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
322011,12800.0,2013-07-11 11:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
322143,12500.0,2013-07-18 23:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
322212,13900.0,2013-07-21 23:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
322260,14000.0,2013-07-24 11:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN


---
---

## v2:

In [14]:
filesout = glob.glob('/raid60/scratch/uli/converted_v2/*')
filesout

['/raid60/scratch/uli/converted_v2/0-20000-0-11019_CEUAS_merged_v0.nc',
 '/raid60/scratch/uli/converted_v2/0-20000-0-22152_CEUAS_merged_v0.nc',
 '/raid60/scratch/uli/converted_v2/0-20000-0-54102_CEUAS_merged_v0.nc']

In [15]:
# with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc') as test:
with eua.CDMDataset(filesout[-1]) as test:

    print(test)
    print(test.recordindices)
    print(test.observations_table)
    print(test.era5fb)
    print(test.era5fb['an_depar@body'][:])
    print(test.era5fb['fg_depar@body'][:])
    aaa = test.to_dataframe(groups=['observations_table', 'era5fb'], variables=['z_coordinate', 'date_time','observed_variable', 'observation_value', 'conversion_flag', 'conversion_method', 'fg_depar@body', 'an_depar@body','biascorr@body', 'biascorr_fg@body'])
aaa

File: <HDF5 file "0-20000-0-54102_CEUAS_merged_v0.nc" (mode r)>
Filename: /raid60/scratch/uli/converted_v2/0-20000-0-54102_CEUAS_merged_v0.nc
(G)roups/(V)ariables: 

 - G | crs__________________________________________ : : 4
 - V | dateindex____________________________________ : : (22851,)
 - G | era5fb_______________________________________ : : 71
 - G | header_table_________________________________ : : 56
 - G | observations_table___________________________ : : 49
 - G | observed_variable____________________________ : : 9
 - G | recordindices________________________________ : : 18
 - G | sensor_configuration_________________________ : : 12
 - G | source_configuration_________________________ : : 2
 - G | station_configuration________________________ : : 48
 - G | station_configuration_codes__________________ : : 7
 - G | station_type_________________________________ : : 4
 - G | units________________________________________ : : 6
 - G | z_coordinate_type____________________________ :

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
0,9473.223633,1964-06-01 11:00:00,0,90000.000000,-2147483648,-2147483648,-131.210556,-170.020813,0.0,0.0
1,9473.223633,1964-07-15 11:00:00,0,88500.000000,-2147483648,-2147483648,-271.075256,-353.135101,0.0,0.0
2,9473.223633,1964-07-30 11:00:00,0,98500.000000,-2147483648,-2147483648,8763.082031,8730.548828,0.0,0.0
3,9473.223633,1964-08-04 23:00:00,0,89300.000000,-2147483648,-2147483648,-244.882065,-286.890900,0.0,0.0
4,9473.223633,1964-08-15 23:00:00,0,89900.000000,-2147483648,-2147483648,-172.731949,-172.108704,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4954084,89810.000000,2019-12-28 23:15:24,140,1.169487,-2147483648,-2147483648,0.506251,0.767653,NaN,NaN
4954085,90780.000000,2019-12-29 11:16:12,140,2.630227,-2147483648,-2147483648,3.134859,2.860141,NaN,NaN
4954086,91280.000000,2019-12-29 23:15:47,140,0.615662,-2147483648,-2147483648,1.913419,2.676003,NaN,NaN
4954087,91770.000000,2019-12-30 11:15:22,140,4.405243,-2147483648,-2147483648,3.490680,3.252057,NaN,NaN


In [16]:
b_aaa = aaa[aaa['conversion_flag'] == 0]
display(b_aaa)
b_aaa[np.isnan(b_aaa['fg_depar@body'])]

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
41,30000.0,1956-08-20 15:00:00,34,9.89,0,3,5.820000,3.810000,NaN,NaN
42,40000.0,1956-08-20 15:00:00,34,9.69,0,3,1.500000,-2.720000,NaN,NaN
43,50000.0,1956-08-20 15:00:00,34,6.95,0,3,-18.830002,-21.860001,NaN,NaN
44,70000.0,1956-08-20 15:00:00,34,1.00,0,3,-5.600000,-8.800000,NaN,NaN
45,85000.0,1956-08-20 15:00:00,34,3.19,0,3,-3.310000,-4.140000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4251393,89810.0,2019-12-28 23:15:24,107,4.00,0,-2147483648,-0.767167,-0.605523,NaN,NaN
4251426,90780.0,2019-12-29 11:16:12,107,6.00,0,-2147483648,1.277103,1.391125,NaN,NaN
4251454,91280.0,2019-12-29 23:15:47,107,1.00,0,-2147483648,-3.204493,-3.804898,NaN,NaN
4251486,91770.0,2019-12-30 11:15:22,107,7.00,0,-2147483648,1.830617,1.644246,NaN,NaN


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
1371,81700.0,1957-06-22 00:00:00,34,5.25,0,3,NaN,4.47,NaN,NaN
1871,49900.0,1957-07-23 00:00:00,34,0.00,0,3,NaN,-0.19,NaN,NaN
1872,50000.0,1957-07-23 00:00:00,34,0.00,0,3,NaN,-0.17,NaN,NaN
1919,89200.0,1957-07-25 00:00:00,34,1.24,0,3,NaN,1.02,NaN,NaN
1951,82700.0,1957-07-27 00:00:00,34,0.00,0,3,NaN,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3700374,89810.0,2019-12-28 23:15:24,106,253.00,0,-2147483648,NaN,NaN,NaN,NaN
3700407,90780.0,2019-12-29 11:16:12,106,244.00,0,-2147483648,NaN,NaN,NaN,NaN
3700435,91280.0,2019-12-29 23:15:47,106,232.00,0,-2147483648,NaN,NaN,NaN,NaN
3700467,91770.0,2019-12-30 11:15:22,106,231.00,0,-2147483648,NaN,NaN,NaN,NaN


In [17]:
aaa[np.isnan(aaa['observation_value'])]

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body


---
---

In [18]:
converted_variables_v1 = 897485 
converted_variables_v2 = 897025 
nan_variables_v1 = 483
nan_variables_v2 = 0

print('gained variables: ', ((converted_variables_v2 - nan_variables_v2) - (converted_variables_v1 - nan_variables_v1)))

time_v1 = 1306.7448754310608 + 185.2546329498291
time_v2 = 1411.9842202663422 + 183.59471774101257
print('more time consumed: ', (time_v2 - time_v1)/60, 'min')

gained variables:  23
more time consumed:  1.7263238271077475 min


---
---

In [19]:
aaa[aaa['observed_variable'] == 34]

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
41,30000.0,1956-08-20 15:00:00,34,9.89,0,3,5.820000,3.810000,NaN,NaN
42,40000.0,1956-08-20 15:00:00,34,9.69,0,3,1.500000,-2.720000,NaN,NaN
43,50000.0,1956-08-20 15:00:00,34,6.95,0,3,-18.830002,-21.860001,NaN,NaN
44,70000.0,1956-08-20 15:00:00,34,1.00,0,3,-5.600000,-8.800000,NaN,NaN
45,85000.0,1956-08-20 15:00:00,34,3.19,0,3,-3.310000,-4.140000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
372761,85000.0,2019-12-30 23:15:09,34,8.59,0,3,-5.809999,-4.520000,NaN,NaN
372762,85500.0,2019-12-30 23:15:09,34,7.59,0,3,-6.000000,-4.690000,NaN,NaN
372763,88100.0,2019-12-30 23:15:09,34,6.79,0,3,-4.110000,-2.970000,NaN,NaN
372764,89100.0,2019-12-30 23:15:09,34,6.89,0,3,-3.880001,-2.830000,NaN,NaN


In [20]:
with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-54102_CEUAS_merged_v0.nc') as test:
    ref = test.to_dataframe(groups=['observations_table', 'era5fb'], variables=['z_coordinate', 'date_time','observed_variable', 'observation_value', 'conversion_flag', 'conversion_method', 'fg_depar@body', 'an_depar@body','biascorr@body', 'biascorr_fg@body'])
    arrayconverter = test.to_dataframe(groups='observations_table', variables=['observed_variable'])
    arrayconverter = arrayconverter.observed_variable.head(1).to_xarray()
ref

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
0,79190.484375,1956-08-20 03:00:00,106,150.000000,-2147483648,-2147483648,NaN,NaN,0.0,0.0
1,79190.484375,1956-08-20 03:00:00,107,7.000000,-2147483648,-2147483648,-0.236686,-0.458606,0.0,0.0
2,79190.484375,1956-08-20 03:00:00,104,-3.500000,-2147483648,-2147483648,-4.902144,-0.307024,0.0,0.0
3,79190.484375,1956-08-20 03:00:00,105,6.062178,-2147483648,-2147483648,-1.037373,-0.678424,0.0,0.0
4,84041.859375,1956-08-20 03:00:00,106,150.000000,-2147483648,-2147483648,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4498261,90840.000000,2019-12-31 11:15:17,117,9845.876953,-2147483648,-2147483648,72.100586,41.154060,NaN,NaN
4498262,90840.000000,2019-12-31 11:15:17,136,253.050003,-2147483648,-2147483648,NaN,NaN,NaN,NaN
4498263,90840.000000,2019-12-31 11:15:17,137,246.750000,-2147483648,-2147483648,NaN,NaN,NaN,NaN
4498264,90840.000000,2019-12-31 11:15:17,138,0.570813,-2147483648,-2147483648,0.296632,0.273383,NaN,NaN


In [21]:
test = ref[ref['z_coordinate'] == 30000.0][ref['date_time'] == '1956-08-20 15:00:00']
display(test)

<ipython-input-21-1dde28cc5541>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test = ref[ref['z_coordinate'] == 30000.0][ref['date_time'] == '1956-08-20 15:00:00']


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
18,30000.0,1956-08-20 15:00:00,117,92868.976562,-2147483648,-2147483648,396.621185,436.673920,0.00000,0.00000
19,30000.0,1956-08-20 15:00:00,85,238.649994,-2147483648,-2147483648,2.755198,2.094388,1.35494,1.35494
20,30000.0,1956-08-20 15:00:00,38,0.360000,-2147483648,-2147483648,-0.295861,-0.172303,0.00000,0.00000


In [22]:
arrayconverter.values = test[test['observed_variable'] == 85]['observation_value']
t = arrayconverter.copy()
arrayconverter.values = test[test['observed_variable'] == 38]['observation_value']
rh = arrayconverter.copy()
arrayconverter.values = test[test['observed_variable'] == 85]['z_coordinate']
zc = arrayconverter.copy()
dpd = rasotools.met.convert.to_dpd(temp=t,press=zc,rel_humi=rh).values
dpd

array([9.89], dtype=float32)

In [23]:
arrayconverter.values = test[test['observed_variable'] == 85]['observation_value'] - test[test['observed_variable'] == 85]['fg_depar@body']
t = arrayconverter.copy()
arrayconverter.values = test[test['observed_variable'] == 38]['observation_value']  - test[test['observed_variable'] == 38]['fg_depar@body']
rh = arrayconverter.copy()
arrayconverter.values = test[test['observed_variable'] == 85]['z_coordinate']
zc = arrayconverter.copy()
dpd_corr = rasotools.met.convert.to_dpd(temp=t,press=zc,rel_humi=rh).values
dpd_corr

array([4.07], dtype=float32)

In [24]:
fg_dep = dpd - dpd_corr
fg_dep

array([5.82], dtype=float32)

In [25]:
arrayconverter.values = test[test['observed_variable'] == 85]['observation_value'] - test[test['observed_variable'] == 85]['an_depar@body']
t = arrayconverter.copy()
arrayconverter.values = test[test['observed_variable'] == 38]['observation_value']  - test[test['observed_variable'] == 38]['an_depar@body']
rh = arrayconverter.copy()
arrayconverter.values = test[test['observed_variable'] == 85]['z_coordinate']
zc = arrayconverter.copy()
dpd_corr = rasotools.met.convert.to_dpd(temp=t,press=zc,rel_humi=rh).values
dpd_corr

array([6.08], dtype=float32)

In [26]:
fg_dep = dpd - dpd_corr
fg_dep

array([3.8100004], dtype=float32)